# Policy Similarity Engine - Enhanced Training Pipeline## 🎯 Enterprise-Grade Similarity Retrieval with Clustering Analysis### Executive Summary- **Business Problem:** Retrieve top 3 most similar historical policies for new business underwriting- **Approach:** Compare multiple clustering algorithms and select optimal similarity engine- **Output:** Production-ready models with full validation and explainability---

## 1. Business Framing### Why Clustering Analysis Before Similarity Engine?**Strategic Approach:**1. **Explore Policy Space** - Understand natural groupings in portfolio2. **Compare Algorithms** - K-Means, DBSCAN, Hierarchical, Gaussian Mixture3. **Evaluate Quality** - Silhouette score, Davies-Bouldin, Calinski-Harabasz4. **Select Best** - Choose optimal algorithm and hyperparameters5. **Build Similarity Engine** - Use insights to construct retrieval system**Why This Matters:**- Validates that policies have meaningful similarity structure- Identifies optimal feature space dimensionality- Informs distance metric selection- Provides interpretable policy segments for business---

In [ ]:
# Environment Setupimport numpy as npimport pandas as pdimport matplotlib.pyplot as pltimport seaborn as snsimport warningsfrom datetime import datetimeimport joblibimport jsonimport osfrom typing import Dict, List, Tuple# Scikit-learnfrom sklearn.preprocessing import StandardScalerfrom sklearn.neighbors import NearestNeighborsfrom sklearn.decomposition import PCAfrom sklearn.manifold import TSNEfrom sklearn.cluster import KMeans, DBSCAN, AgglomerativeClusteringfrom sklearn.mixture import GaussianMixturefrom sklearn.metrics import (    silhouette_score,     silhouette_samples,    davies_bouldin_score,    calinski_harabasz_score)# Advanced visualizationfrom mpl_toolkits.mplot3d import Axes3D# UMAP for better visualizationtry:    import umap    UMAP_AVAILABLE = Trueexcept ImportError:    UMAP_AVAILABLE = False    print("⚠️ UMAP not available. Install with: pip install umap-learn")# Sentence transformerstry:    from sentence_transformers import SentenceTransformer    SENTENCE_TRANSFORMER_AVAILABLE = Trueexcept ImportError:    SENTENCE_TRANSFORMER_AVAILABLE = False    print("⚠️ sentence-transformers not available")# FAISStry:    import faiss    FAISS_AVAILABLE = Trueexcept ImportError:    FAISS_AVAILABLE = False    print("⚠️ FAISS not available")# SHAP for explainabilitytry:    import shap    SHAP_AVAILABLE = Trueexcept ImportError:    SHAP_AVAILABLE = False    print("⚠️ SHAP not available. Install with: pip install shap")# Configurationwarnings.filterwarnings('ignore')plt.style.use('seaborn-v0_8-darkgrid')sns.set_palette("husl")RANDOM_SEED = 42np.random.seed(RANDOM_SEED)print("✓ Environment ready")

## 2. Data Loading

In [ ]:
# Load dataDATA_PATH = 'insurance_policies.csv'try:    df = pd.read_csv(DATA_PATH, low_memory=False)    print(f"✓ Data loaded: {df.shape}")except FileNotFoundError:    print("Creating synthetic data for demonstration...")    n = 5000    df = pd.DataFrame({        'System Reference Number': [f'SRN{i:06d}' for i in range(n)],        'Policy Number': [f'POL{i:06d}' if i%10!=0 else np.nan for i in range(n)],        'Effective Date': pd.date_range('2020-01-01', periods=n, freq='H'),        'Expiration Date': pd.date_range('2021-01-01', periods=n, freq='H'),        'DUNS_NUMBER_1': np.random.randint(100000, 999999, n),        'policy_tiv': np.random.lognormal(15, 1.5, n),        'Revenue': np.random.lognormal(16, 2, n),        'highest_location_tiv': np.random.lognormal(14, 1.2, n),        'POSTAL_CD': np.random.choice(range(10000, 99999), n),        'LAT_NEW': np.random.uniform(25, 49, n),        'LATIT': np.random.uniform(25, 49, n),        'LONGIT': np.random.uniform(-125, -70, n),        'LONG_NEW': np.random.uniform(-125, -70, n),        'SIC_1': np.random.choice([1234, 2345, 3456, 4567, 5678, 6789], n),        'EMP_TOT': np.random.lognormal(4, 2, n),        'SLES_VOL': np.random.lognormal(15, 1.8, n),        'YR_STRT': np.random.choice(range(1970, 2020), n),        'STAT_IND': np.random.choice([0, 1], n),        'SUBS_IND': np.random.choice([0, 1], n),        'outliers': np.random.choice([0, 1], n, p=[0.95, 0.05]),        '2012 NAIC Code': np.random.choice(['524126', '524210', '524292'], n),        '2012 NAIC Description': np.random.choice(['Property Insurance', 'Casualty Insurance', 'Liability Insurance'], n),        'Programme Type': np.random.choice(['Corporate', 'SME', 'Enterprise'], n),        'Portfolio Class Code': np.random.choice(['PC01', 'PC02', 'PC03'], n),        'Portfolio Segmentation': np.random.choice(['Manufacturing', 'Retail', 'Technology', 'Healthcare'], n),        'Producer Code': np.random.choice([f'PR{i:03d}' for i in range(50)], n),        'Producer': np.random.choice([f'Producer_{i}' for i in range(50)], n),        'LOCATION': np.random.choice(['New York', 'California', 'Texas', 'Florida'], n),        'COMPANY_NAME': [f'Company_{i%500}' for i in range(n)],        'ADDR': [f'{i} Main St' for i in range(n)],        'Product': np.random.choice(['Property', 'Casualty', 'Liability', 'Workers Comp'], n),        'Sub Product': np.random.choice(['Standard', 'Premium', 'Basic'], n),        '[Process Type]': np.random.choice(['New', 'Renewal', 'Endorsement'], n),        'Position Type': np.random.choice(['Primary', 'Excess'], n),        'Placement Type': np.random.choice(['Direct', 'Reinsurance'], n),        'Practice/Non-Practice': np.random.choice(['Practice', 'Non-Practice'], n),        'Revised Practice': np.random.choice(['Yes', 'No'], n),        'Policy Holder Name': [f'Holder_{i%800}' for i in range(n)],        'Policy Industry Code': np.random.choice(['31', '32', '33', '44', '45'], n),        'Policy Industry Description': np.random.choice(            ['Manufacturing - Food', 'Manufacturing - Chemical', 'Retail Trade', 'Technology Services'],            n        ),        '2012 NAIC 2 Digit Code': np.random.choice(['52', '53', '54'], n),        '2012 NAIC 2 Digit Description': np.random.choice(['Insurance', 'Real Estate'], n),        '2012 NAIC 3 Digit Code': np.random.choice(['524', '525', '531'], n),        '2012 NAIC 3 Digit Description': np.random.choice(['Insurance Carriers', 'Insurance Agencies'], n),        '2012 NAIC 4 Digit Code': np.random.choice(['5241', '5242', '5311'], n),        '2012 NAIC 4 Digit Description': np.random.choice(['Direct Insurance', 'Reinsurance'], n),        'Short Tail / Long Tail': np.random.choice(['Short Tail', 'Long Tail'], n)    })    print(f"✓ Synthetic data created: {df.shape}")print(f"Rows: {df.shape[0]:,} | Columns: {df.shape[1]}")df.head(3)

## 3. Data Cleaning & Feature Engineering

In [ ]:
# Store identifiersidentifiers = df[['System Reference Number']].copy()if 'Policy Number' in df.columns:    identifiers['Policy Number'] = df['Policy Number']# Remove identifiersdf_clean = df.drop(columns=['System Reference Number', 'DUNS_NUMBER_1', 'Policy Number'], errors='ignore')print(f"✓ Identifiers stored: {len(identifiers)}")print(f"✓ Remaining features: {df_clean.shape[1]}")

In [ ]:
# Date Feature Engineeringdef extract_date_features(df):    if 'Effective Date' in df.columns:        df['Effective Date'] = pd.to_datetime(df['Effective Date'], errors='coerce')        df['Expiration Date'] = pd.to_datetime(df['Expiration Date'], errors='coerce')                df['policy_tenure_days'] = (df['Expiration Date'] - df['Effective Date']).dt.days        df['effective_month'] = df['Effective Date'].dt.month        df['effective_quarter'] = df['Effective Date'].dt.quarter        df['effective_year'] = df['Effective Date'].dt.year                # Cyclical encoding        df['month_sin'] = np.sin(2 * np.pi * df['effective_month'] / 12)        df['month_cos'] = np.cos(2 * np.pi * df['effective_month'] / 12)        df['quarter_sin'] = np.sin(2 * np.pi * df['effective_quarter'] / 4)        df['quarter_cos'] = np.cos(2 * np.pi * df['effective_quarter'] / 4)                df = df.drop(columns=['Effective Date', 'Expiration Date'])    return dfdf_clean = extract_date_features(df_clean)print("✓ Date features extracted")

In [ ]:
# Geospatial Featuresdef haversine(lat1, lon1, lat2, lon2):    '''Calculate haversine distance in km'''    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])    dlat, dlon = lat2 - lat1, lon2 - lon1    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2    return 2 * 6371 * np.arcsin(np.sqrt(a))# Consolidate lat/longif 'LAT_NEW' in df_clean.columns and 'LATIT' in df_clean.columns:    df_clean['latitude'] = df_clean[['LAT_NEW', 'LATIT']].mean(axis=1)    df_clean['longitude'] = df_clean[['LONG_NEW', 'LONGIT']].mean(axis=1)    df_clean = df_clean.drop(columns=['LAT_NEW', 'LATIT', 'LONG_NEW', 'LONGIT'])# Distance from major hubsif 'latitude' in df_clean.columns:    NYC_LAT, NYC_LON = 40.7128, -74.0060    LA_LAT, LA_LON = 34.0522, -118.2437        df_clean['dist_from_nyc_km'] = haversine(        df_clean['latitude'].fillna(NYC_LAT),        df_clean['longitude'].fillna(NYC_LON),        NYC_LAT, NYC_LON    )        df_clean['dist_from_la_km'] = haversine(        df_clean['latitude'].fillna(LA_LAT),        df_clean['longitude'].fillna(LA_LON),        LA_LAT, LA_LON    )        # Geographic region encoding    df_clean['geo_region'] = 'Other'    df_clean.loc[(df_clean['latitude'] >= 37) & (df_clean['longitude'] >= -95), 'geo_region'] = 'Northeast'    df_clean.loc[(df_clean['latitude'] >= 37) & (df_clean['longitude'] < -95), 'geo_region'] = 'Northwest'    df_clean.loc[(df_clean['latitude'] < 37) & (df_clean['longitude'] >= -95), 'geo_region'] = 'Southeast'    df_clean.loc[(df_clean['latitude'] < 37) & (df_clean['longitude'] < -95), 'geo_region'] = 'Southwest'print("✓ Geospatial features created")

In [ ]:
# Categorical Handlingdef group_rare_categories(df, col, threshold=0.01):    if col not in df.columns:        return df    value_counts = df[col].value_counts(normalize=True)    rare = value_counts[value_counts < threshold].index.tolist()    if rare:        df[col] = df[col].replace(rare, 'Other')        print(f"  {col}: {len(rare)} rare categories → 'Other'")    return dfcategorical_cols = df_clean.select_dtypes(include=['object']).columns.tolist()for col in categorical_cols:    if df_clean[col].nunique() > 50:        df_clean = group_rare_categories(df_clean, col, threshold=0.01)print("✓ Rare categories grouped")

In [ ]:
# Missing Value Imputationnumerical_cols = df_clean.select_dtypes(include=[np.number]).columns.tolist()for col in numerical_cols:    if df_clean[col].isnull().sum() > 0:        df_clean[col].fillna(df_clean[col].median(), inplace=True)for col in categorical_cols:    if df_clean[col].isnull().sum() > 0:        df_clean[col].fillna('Missing', inplace=True)print(f"✓ Missing values handled")print(f"✓ Final clean shape: {df_clean.shape}")

## 4. Feature Encoding

In [ ]:
# Separate feature typespure_numerical = [    c for c in ['policy_tiv', 'Revenue', 'highest_location_tiv', 'EMP_TOT', 'SLES_VOL',                'latitude', 'longitude', 'dist_from_nyc_km', 'dist_from_la_km',                'policy_tenure_days', 'month_sin', 'month_cos', 'quarter_sin', 'quarter_cos',                'YR_STRT', 'effective_year', 'effective_month', 'effective_quarter']     if c in df_clean.columns]low_cardinality = []high_cardinality = []for col in categorical_cols:    if col not in df_clean.columns:        continue    nunique = df_clean[col].nunique()    if nunique < 20:        low_cardinality.append(col)    else:        high_cardinality.append(col)text_fields = [    c for c in ['Policy Industry Description', '2012 NAIC Description', 'Portfolio Segmentation']     if c in df_clean.columns]print(f"Pure Numerical: {len(pure_numerical)}")print(f"Low Cardinality: {len(low_cardinality)}")print(f"High Cardinality: {len(high_cardinality)}")print(f"Text Fields: {len(text_fields)}")

In [ ]:
# Text Embeddingstext_embeddings = {}if SENTENCE_TRANSFORMER_AVAILABLE and text_fields:    model = SentenceTransformer('all-MiniLM-L6-v2')        for col in text_fields:        if col in df_clean.columns:            print(f"Embedding: {col}")            texts = df_clean[col].fillna('').astype(str).tolist()            embeddings = model.encode(texts, show_progress_bar=True, batch_size=32)            text_embeddings[col] = embeddings                        emb_df = pd.DataFrame(                embeddings,                 columns=[f'{col}_emb_{i}' for i in range(embeddings.shape[1])]            )            df_clean = pd.concat([df_clean, emb_df], axis=1)        print(f"✓ Text embeddings: {len(text_embeddings)} fields")else:    print("⚠️ Text embeddings skipped")

In [ ]:
# Encodingdf_encoded = df_clean.copy()# One-hot encodingif low_cardinality:    df_encoded = pd.get_dummies(df_encoded, columns=low_cardinality, drop_first=True)    print(f"✓ One-hot encoded: {len(low_cardinality)} features")# Frequency encodingfrequency_encodings = {}for col in high_cardinality:    if col in df_clean.columns:        freq_map = df_clean[col].value_counts(normalize=True).to_dict()        frequency_encodings[col] = freq_map        df_encoded[f'{col}_freq'] = df_clean[col].map(freq_map).fillna(0)        df_encoded = df_encoded.drop(columns=[col])if high_cardinality:    print(f"✓ Frequency encoded: {len(high_cardinality)} features")print(f"✓ Encoded shape: {df_encoded.shape}")

In [ ]:
# Scalingnumerical_features = df_encoded.select_dtypes(include=[np.number]).columns.tolist()scaler = StandardScaler()df_scaled = df_encoded.copy()df_scaled[numerical_features] = scaler.fit_transform(df_encoded[numerical_features])print(f"✓ Scaled {len(numerical_features)} features")print(f"✓ Final shape: {df_scaled.shape}")

## 5. ✅ VALIDATION: Feature Matrix Ready for Clustering### Pre-Clustering Validation Checks

In [ ]:
def validate_feature_matrix(df_scaled):    '''Comprehensive validation before clustering'''        validation_results = {        'checks_passed': [],        'checks_failed': [],        'warnings': []    }        print("="*80)    print("FEATURE MATRIX VALIDATION")    print("="*80)        # Check 1: All numeric    non_numeric = df_scaled.select_dtypes(exclude=[np.number]).columns.tolist()    if len(non_numeric) == 0:        print("✓ CHECK 1 PASSED: All features are numeric")        validation_results['checks_passed'].append('all_numeric')    else:        print(f"✗ CHECK 1 FAILED: {len(non_numeric)} non-numeric columns found")        print(f"  Columns: {non_numeric}")        validation_results['checks_failed'].append('non_numeric_found')        # Check 2: No missing values    missing_count = df_scaled.isnull().sum().sum()    if missing_count == 0:        print("✓ CHECK 2 PASSED: No missing values")        validation_results['checks_passed'].append('no_missing')    else:        print(f"✗ CHECK 2 FAILED: {missing_count} missing values found")        validation_results['checks_failed'].append('missing_values')        # Check 3: No infinite values    inf_count = np.isinf(df_scaled.select_dtypes(include=[np.number]).values).sum()    if inf_count == 0:        print("✓ CHECK 3 PASSED: No infinite values")        validation_results['checks_passed'].append('no_infinite')    else:        print(f"✗ CHECK 3 FAILED: {inf_count} infinite values found")        validation_results['checks_failed'].append('infinite_values')        # Check 4: Scaled (mean ≈ 0, std ≈ 1)    means = df_scaled.mean()    stds = df_scaled.std()        mean_check = (means.abs() < 0.1).sum() / len(means)    std_check = ((stds - 1).abs() < 0.1).sum() / len(stds)        if mean_check > 0.8 and std_check > 0.8:        print(f"✓ CHECK 4 PASSED: Features properly scaled")        print(f"  Mean ≈ 0: {mean_check:.1%} of features")        print(f"  Std ≈ 1: {std_check:.1%} of features")        validation_results['checks_passed'].append('properly_scaled')    else:        print(f"⚠ CHECK 4 WARNING: Scaling may need review")        print(f"  Mean ≈ 0: {mean_check:.1%} of features")        print(f"  Std ≈ 1: {std_check:.1%} of features")        validation_results['warnings'].append('scaling_review')        # Check 5: Variance check    zero_var_cols = df_scaled.columns[df_scaled.std() == 0].tolist()    if len(zero_var_cols) == 0:        print("✓ CHECK 5 PASSED: All features have variance")        validation_results['checks_passed'].append('has_variance')    else:        print(f"⚠ CHECK 5 WARNING: {len(zero_var_cols)} zero-variance features")        print(f"  Columns: {zero_var_cols[:5]}")        validation_results['warnings'].append('zero_variance')        # Check 6: Shape check    print(f"\n✓ CHECK 6 PASSED: Shape validation")    print(f"  Rows (policies): {df_scaled.shape[0]:,}")    print(f"  Columns (features): {df_scaled.shape[1]:,}")    validation_results['checks_passed'].append('shape_valid')        # Check 7: Data type consistency    dtypes = df_scaled.dtypes.value_counts()    print(f"\n✓ CHECK 7 PASSED: Data types")    for dtype, count in dtypes.items():        print(f"  {dtype}: {count} columns")    validation_results['checks_passed'].append('dtypes_consistent')        # Summary    print("\n" + "="*80)    print("VALIDATION SUMMARY")    print("="*80)    print(f"✓ Checks Passed: {len(validation_results['checks_passed'])}")    print(f"✗ Checks Failed: {len(validation_results['checks_failed'])}")    print(f"⚠ Warnings: {len(validation_results['warnings'])}")        if len(validation_results['checks_failed']) == 0:        print("\n🎉 ALL CRITICAL CHECKS PASSED - READY FOR CLUSTERING")        return True, validation_results    else:        print("\n⚠️ SOME CHECKS FAILED - REVIEW REQUIRED")        return False, validation_results# Run validationis_valid, validation_results = validate_feature_matrix(df_scaled)

In [ ]:
# Additional diagnosticsprint("\n" + "="*80)print("FEATURE STATISTICS")print("="*80)stats_df = pd.DataFrame({    'Feature': df_scaled.columns,    'Mean': df_scaled.mean().values,    'Std': df_scaled.std().values,    'Min': df_scaled.min().values,    'Max': df_scaled.max().values,    'Skewness': df_scaled.skew().values})print("\nTop 10 Features by Std Dev:")print(stats_df.nlargest(10, 'Std')[['Feature', 'Std']])print("\nTop 10 Features by Skewness:")print(stats_df.nlargest(10, 'Skewness')[['Feature', 'Skewness']])

In [ ]:
# Correlation analysisprint("\n" + "="*80)print("CORRELATION ANALYSIS")print("="*80)# Sample for faster computationsample_size = min(1000, len(df_scaled))sample_idx = np.random.choice(len(df_scaled), sample_size, replace=False)df_sample = df_scaled.iloc[sample_idx]# Compute correlation matrixcorr_matrix = df_sample.corr()# Find highly correlated pairshigh_corr = []for i in range(len(corr_matrix.columns)):    for j in range(i+1, len(corr_matrix.columns)):        if abs(corr_matrix.iloc[i, j]) > 0.9:            high_corr.append((                corr_matrix.columns[i],                corr_matrix.columns[j],                corr_matrix.iloc[i, j]            ))if high_corr:    print(f"\n⚠️ Found {len(high_corr)} highly correlated feature pairs (|r| > 0.9):")    for feat1, feat2, corr_val in high_corr[:10]:        print(f"  {feat1} <-> {feat2}: {corr_val:.3f}")else:    print("\n✓ No highly correlated features found (|r| > 0.9)")

## 6. Dimensionality Reduction Analysis### PCA for Variance Understanding

In [ ]:
# PCA Analysisn_components_pca = min(50, df_scaled.shape[1], df_scaled.shape[0])pca = PCA(n_components=n_components_pca, random_state=RANDOM_SEED)X_pca = pca.fit_transform(df_scaled)cumsum_variance = np.cumsum(pca.explained_variance_ratio_)n_components_95 = np.argmax(cumsum_variance >= 0.95) + 1n_components_90 = np.argmax(cumsum_variance >= 0.90) + 1print("="*80)print("PCA DIMENSIONALITY ANALYSIS")print("="*80)print(f"Original dimensions: {df_scaled.shape[1]}")print(f"Components for 90% variance: {n_components_90}")print(f"Components for 95% variance: {n_components_95}")print(f"Components for 99% variance: {np.argmax(cumsum_variance >= 0.99) + 1}")# Visualizefig, axes = plt.subplots(1, 2, figsize=(15, 5))# Scree plotaxes[0].plot(range(1, len(pca.explained_variance_ratio_)+1),              pca.explained_variance_ratio_, marker='o', linewidth=2)axes[0].axhline(y=0.05, color='r', linestyle='--', alpha=0.7, label='5% threshold')axes[0].set_xlabel('Principal Component', fontsize=12)axes[0].set_ylabel('Explained Variance Ratio', fontsize=12)axes[0].set_title('PCA Scree Plot', fontsize=14, fontweight='bold')axes[0].legend()axes[0].grid(True, alpha=0.3)# Cumulative varianceaxes[1].plot(range(1, len(cumsum_variance)+1), cumsum_variance,              marker='o', linewidth=2, color='green')axes[1].axhline(y=0.90, color='orange', linestyle='--', alpha=0.7, label='90%')axes[1].axhline(y=0.95, color='red', linestyle='--', alpha=0.7, label='95%')axes[1].axvline(x=n_components_95, color='purple', linestyle='--', alpha=0.7,                 label=f'{n_components_95} components')axes[1].set_xlabel('Number of Components', fontsize=12)axes[1].set_ylabel('Cumulative Explained Variance', fontsize=12)axes[1].set_title('Cumulative Variance Explained', fontsize=14, fontweight='bold')axes[1].legend()axes[1].grid(True, alpha=0.3)plt.tight_layout()plt.savefig('/home/claude/pca_analysis.png', dpi=300, bbox_inches='tight')plt.show()print(f"\n✓ PCA plot saved")

In [ ]:
# Prepare reduced datasets for clusteringX_full = df_scaled.valuesX_pca_90 = X_pca[:, :n_components_90]X_pca_95 = X_pca[:, :n_components_95]print(f"\n✓ Prepared feature matrices:")print(f"  Full: {X_full.shape}")print(f"  PCA 90%: {X_pca_90.shape}")print(f"  PCA 95%: {X_pca_95.shape}")

## 7. 🔬 CLUSTERING ANALYSIS### Compare Multiple Algorithms with Different Hyperparameters

In [ ]:
def evaluate_clustering(X, labels, algorithm_name, params):    '''Calculate comprehensive clustering metrics'''        # Remove noise points for metrics (label = -1 in DBSCAN)    mask = labels != -1    X_clean = X[mask]    labels_clean = labels[mask]        n_clusters = len(np.unique(labels_clean))    n_noise = np.sum(labels == -1)        metrics = {        'algorithm': algorithm_name,        'params': params,        'n_clusters': n_clusters,        'n_noise': n_noise,        'noise_pct': (n_noise / len(labels) * 100) if len(labels) > 0 else 0    }        # Only calculate metrics if we have enough clusters and samples    if n_clusters > 1 and len(X_clean) > n_clusters:        try:            metrics['silhouette'] = silhouette_score(X_clean, labels_clean)            metrics['davies_bouldin'] = davies_bouldin_score(X_clean, labels_clean)            metrics['calinski_harabasz'] = calinski_harabasz_score(X_clean, labels_clean)        except:            metrics['silhouette'] = np.nan            metrics['davies_bouldin'] = np.nan            metrics['calinski_harabasz'] = np.nan    else:        metrics['silhouette'] = np.nan        metrics['davies_bouldin'] = np.nan        metrics['calinski_harabasz'] = np.nan        return metricsprint("✓ Evaluation function defined")

In [ ]:
# EXPERIMENT 1: K-Means with different Kprint("="*80)print("EXPERIMENT 1: K-MEANS CLUSTERING")print("="*80)kmeans_results = []k_values = [3, 4, 5, 6, 7, 8, 10, 12, 15, 20]for k in k_values:    print(f"\nTesting K-Means with K={k}...")        kmeans = KMeans(n_clusters=k, random_state=RANDOM_SEED, n_init=10)    labels = kmeans.fit_predict(X_pca_95)  # Use PCA 95% features        metrics = evaluate_clustering(X_pca_95, labels, 'K-Means', {'k': k})    kmeans_results.append(metrics)        print(f"  Silhouette: {metrics['silhouette']:.4f}")    print(f"  Davies-Bouldin: {metrics['davies_bouldin']:.4f}")    print(f"  Calinski-Harabasz: {metrics['calinski_harabasz']:.2f}")# Convert to DataFramekmeans_df = pd.DataFrame(kmeans_results)print("\n" + "="*80)print("K-MEANS RESULTS SUMMARY")print("="*80)print(kmeans_df.to_string(index=False))

In [ ]:
# Visualize K-Means metricsfig, axes = plt.subplots(1, 3, figsize=(18, 5))# Silhouette Scoreaxes[0].plot(kmeans_df['params'].apply(lambda x: x['k']),              kmeans_df['silhouette'], marker='o', linewidth=2, markersize=8)axes[0].set_xlabel('Number of Clusters (K)', fontsize=12)axes[0].set_ylabel('Silhouette Score', fontsize=12)axes[0].set_title('K-Means: Silhouette Score\n(Higher is Better)', fontsize=14, fontweight='bold')axes[0].grid(True, alpha=0.3)# Davies-Bouldin Indexaxes[1].plot(kmeans_df['params'].apply(lambda x: x['k']),              kmeans_df['davies_bouldin'], marker='s', linewidth=2, markersize=8, color='orange')axes[1].set_xlabel('Number of Clusters (K)', fontsize=12)axes[1].set_ylabel('Davies-Bouldin Index', fontsize=12)axes[1].set_title('K-Means: Davies-Bouldin Index\n(Lower is Better)', fontsize=14, fontweight='bold')axes[1].grid(True, alpha=0.3)# Calinski-Harabasz Scoreaxes[2].plot(kmeans_df['params'].apply(lambda x: x['k']),              kmeans_df['calinski_harabasz'], marker='^', linewidth=2, markersize=8, color='green')axes[2].set_xlabel('Number of Clusters (K)', fontsize=12)axes[2].set_ylabel('Calinski-Harabasz Score', fontsize=12)axes[2].set_title('K-Means: Calinski-Harabasz Score\n(Higher is Better)', fontsize=14, fontweight='bold')axes[2].grid(True, alpha=0.3)plt.tight_layout()plt.savefig('/home/claude/kmeans_metrics.png', dpi=300, bbox_inches='tight')plt.show()print("✓ K-Means metrics visualization saved")

In [ ]:
# EXPERIMENT 2: Hierarchical Clusteringprint("\n" + "="*80)print("EXPERIMENT 2: HIERARCHICAL CLUSTERING")print("="*80)hierarchical_results = []linkage_methods = ['ward', 'complete', 'average']n_clusters_list = [5, 7, 10]for linkage in linkage_methods:    for n_clust in n_clusters_list:        print(f"\nTesting Hierarchical (linkage={linkage}, n_clusters={n_clust})...")                hierarchical = AgglomerativeClustering(n_clusters=n_clust, linkage=linkage)        labels = hierarchical.fit_predict(X_pca_95)                metrics = evaluate_clustering(            X_pca_95, labels,             'Hierarchical',             {'linkage': linkage, 'n_clusters': n_clust}        )        hierarchical_results.append(metrics)                print(f"  Silhouette: {metrics['silhouette']:.4f}")        print(f"  Davies-Bouldin: {metrics['davies_bouldin']:.4f}")hierarchical_df = pd.DataFrame(hierarchical_results)print("\n" + "="*80)print("HIERARCHICAL RESULTS SUMMARY")print("="*80)print(hierarchical_df.to_string(index=False))

In [ ]:
# EXPERIMENT 3: DBSCANprint("\n" + "="*80)print("EXPERIMENT 3: DBSCAN CLUSTERING")print("="*80)dbscan_results = []eps_values = [0.5, 1.0, 1.5, 2.0, 3.0]min_samples_values = [5, 10, 20]for eps in eps_values:    for min_samples in min_samples_values:        print(f"\nTesting DBSCAN (eps={eps}, min_samples={min_samples})...")                dbscan = DBSCAN(eps=eps, min_samples=min_samples, n_jobs=-1)        labels = dbscan.fit_predict(X_pca_95)                metrics = evaluate_clustering(            X_pca_95, labels,            'DBSCAN',            {'eps': eps, 'min_samples': min_samples}        )        dbscan_results.append(metrics)                print(f"  Clusters found: {metrics['n_clusters']}")        print(f"  Noise points: {metrics['n_noise']} ({metrics['noise_pct']:.1f}%)")        if not np.isnan(metrics['silhouette']):            print(f"  Silhouette: {metrics['silhouette']:.4f}")dbscan_df = pd.DataFrame(dbscan_results)print("\n" + "="*80)print("DBSCAN RESULTS SUMMARY")print("="*80)print(dbscan_df[dbscan_df['n_clusters'] > 1].to_string(index=False))

In [ ]:
# EXPERIMENT 4: Gaussian Mixture Modelsprint("\n" + "="*80)print("EXPERIMENT 4: GAUSSIAN MIXTURE MODELS")print("="*80)gmm_results = []n_components_list = [3, 5, 7, 10, 12]covariance_types = ['full', 'tied', 'diag']for n_comp in n_components_list:    for cov_type in covariance_types:        print(f"\nTesting GMM (n_components={n_comp}, covariance={cov_type})...")                try:            gmm = GaussianMixture(                n_components=n_comp,                 covariance_type=cov_type,                 random_state=RANDOM_SEED,                n_init=3            )            labels = gmm.fit_predict(X_pca_95)                        metrics = evaluate_clustering(                X_pca_95, labels,                'GMM',                {'n_components': n_comp, 'covariance_type': cov_type}            )            metrics['bic'] = gmm.bic(X_pca_95)            metrics['aic'] = gmm.aic(X_pca_95)                        gmm_results.append(metrics)                        print(f"  Silhouette: {metrics['silhouette']:.4f}")            print(f"  BIC: {metrics['bic']:.2f}")            print(f"  AIC: {metrics['aic']:.2f}")        except Exception as e:            print(f"  Failed: {str(e)}")gmm_df = pd.DataFrame(gmm_results)print("\n" + "="*80)print("GMM RESULTS SUMMARY")print("="*80)print(gmm_df.to_string(index=False))

## 8. Comprehensive Evaluation & Model Selection

In [ ]:
# Combine all resultsall_results = pd.concat([    kmeans_df,    hierarchical_df,    dbscan_df[dbscan_df['n_clusters'] > 1],  # Only valid DBSCAN results    gmm_df], ignore_index=True)# Rank by metricsprint("="*80)print("TOP 10 MODELS BY SILHOUETTE SCORE")print("="*80)top_silhouette = all_results.nlargest(10, 'silhouette')[    ['algorithm', 'params', 'n_clusters', 'silhouette', 'davies_bouldin', 'calinski_harabasz']]print(top_silhouette.to_string(index=False))print("\n" + "="*80)print("TOP 10 MODELS BY DAVIES-BOULDIN (Lower is Better)")print("="*80)top_db = all_results.nsmallest(10, 'davies_bouldin')[    ['algorithm', 'params', 'n_clusters', 'silhouette', 'davies_bouldin', 'calinski_harabasz']]print(top_db.to_string(index=False))print("\n" + "="*80)print("TOP 10 MODELS BY CALINSKI-HARABASZ SCORE")print("="*80)top_ch = all_results.nlargest(10, 'calinski_harabasz')[    ['algorithm', 'params', 'n_clusters', 'silhouette', 'davies_bouldin', 'calinski_harabasz']]print(top_ch.to_string(index=False))

In [ ]:
# Select best model based on combined metrics# Normalize scores (0-1 scale)all_results['silhouette_norm'] = (all_results['silhouette'] - all_results['silhouette'].min()) / \                                   (all_results['silhouette'].max() - all_results['silhouette'].min())all_results['davies_bouldin_norm'] = 1 - ((all_results['davies_bouldin'] - all_results['davies_bouldin'].min()) / \                                           (all_results['davies_bouldin'].max() - all_results['davies_bouldin'].min()))all_results['calinski_harabasz_norm'] = (all_results['calinski_harabasz'] - all_results['calinski_harabasz'].min()) / \                                         (all_results['calinski_harabasz'].max() - all_results['calinski_harabasz'].min())# Combined score (weighted average)all_results['combined_score'] = (    0.4 * all_results['silhouette_norm'] +    0.3 * all_results['davies_bouldin_norm'] +    0.3 * all_results['calinski_harabasz_norm'])print("\n" + "="*80)print("TOP 10 MODELS BY COMBINED SCORE")print("="*80)top_combined = all_results.nlargest(10, 'combined_score')[    ['algorithm', 'params', 'n_clusters', 'silhouette', 'davies_bouldin',      'calinski_harabasz', 'combined_score']]print(top_combined.to_string(index=False))# Select best modelbest_model_idx = all_results['combined_score'].idxmax()best_model_info = all_results.loc[best_model_idx]print("\n" + "="*80)print("🏆 BEST MODEL SELECTED")print("="*80)print(f"Algorithm: {best_model_info['algorithm']}")print(f"Parameters: {best_model_info['params']}")print(f"Number of Clusters: {best_model_info['n_clusters']}")print(f"Silhouette Score: {best_model_info['silhouette']:.4f}")print(f"Davies-Bouldin Index: {best_model_info['davies_bouldin']:.4f}")print(f"Calinski-Harabasz Score: {best_model_info['calinski_harabasz']:.2f}")print(f"Combined Score: {best_model_info['combined_score']:.4f}")

## 9. Train Final Model with Best Configuration

In [ ]:
# Train final model with best parametersalgorithm = best_model_info['algorithm']params = best_model_info['params']print(f"Training final {algorithm} model...")if algorithm == 'K-Means':    final_model = KMeans(        n_clusters=params['k'],        random_state=RANDOM_SEED,        n_init=20  # More iterations for final model    )    final_labels = final_model.fit_predict(X_pca_95)elif algorithm == 'Hierarchical':    final_model = AgglomerativeClustering(        n_clusters=params['n_clusters'],        linkage=params['linkage']    )    final_labels = final_model.fit_predict(X_pca_95)elif algorithm == 'DBSCAN':    final_model = DBSCAN(        eps=params['eps'],        min_samples=params['min_samples'],        n_jobs=-1    )    final_labels = final_model.fit_predict(X_pca_95)elif algorithm == 'GMM':    final_model = GaussianMixture(        n_components=params['n_components'],        covariance_type=params['covariance_type'],        random_state=RANDOM_SEED,        n_init=10    )    final_labels = final_model.fit_predict(X_pca_95)print(f"✓ Final model trained")print(f"✓ Cluster labels assigned: {len(final_labels)}")

## 10. Cluster Visualization (2D & 3D)

In [ ]:
# 2D Visualization with UMAP or t-SNEprint("Generating 2D projections...")# t-SNE (slower but good for local structure)tsne = TSNE(n_components=2, random_state=RANDOM_SEED, perplexity=30)X_tsne = tsne.fit_transform(X_pca_95[:min(2000, len(X_pca_95))])  # Sample for speedlabels_tsne = final_labels[:min(2000, len(final_labels))]# UMAP (faster, preserves global structure)if UMAP_AVAILABLE:    umap_model = umap.UMAP(n_components=2, random_state=RANDOM_SEED, n_neighbors=15)    X_umap = umap_model.fit_transform(X_pca_95)        fig, axes = plt.subplots(1, 2, figsize=(20, 8))        # t-SNE plot    scatter1 = axes[0].scatter(X_tsne[:, 0], X_tsne[:, 1],                                c=labels_tsne, cmap='tab20',                                alpha=0.6, s=50, edgecolors='k', linewidth=0.5)    axes[0].set_xlabel('t-SNE Dimension 1', fontsize=12)    axes[0].set_ylabel('t-SNE Dimension 2', fontsize=12)    axes[0].set_title(f't-SNE Projection of {algorithm} Clusters', fontsize=14, fontweight='bold')    plt.colorbar(scatter1, ax=axes[0], label='Cluster')        # UMAP plot    scatter2 = axes[1].scatter(X_umap[:, 0], X_umap[:, 1],                                c=final_labels, cmap='tab20',                                alpha=0.6, s=50, edgecolors='k', linewidth=0.5)    axes[1].set_xlabel('UMAP Dimension 1', fontsize=12)    axes[1].set_ylabel('UMAP Dimension 2', fontsize=12)    axes[1].set_title(f'UMAP Projection of {algorithm} Clusters', fontsize=14, fontweight='bold')    plt.colorbar(scatter2, ax=axes[1], label='Cluster')else:    fig, ax = plt.subplots(1, 1, figsize=(12, 8))    scatter = ax.scatter(X_tsne[:, 0], X_tsne[:, 1],                         c=labels_tsne, cmap='tab20',                         alpha=0.6, s=50, edgecolors='k', linewidth=0.5)    ax.set_xlabel('t-SNE Dimension 1', fontsize=12)    ax.set_ylabel('t-SNE Dimension 2', fontsize=12)    ax.set_title(f't-SNE Projection of {algorithm} Clusters', fontsize=14, fontweight='bold')    plt.colorbar(scatter, ax=ax, label='Cluster')plt.tight_layout()plt.savefig('/home/claude/cluster_2d_visualization.png', dpi=300, bbox_inches='tight')plt.show()print("✓ 2D visualization saved")

In [ ]:
# 3D Visualization with PCAprint("Generating 3D projection...")fig = plt.figure(figsize=(14, 10))ax = fig.add_subplot(111, projection='3d')# Use first 3 PCA componentsscatter = ax.scatter(X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],                     c=final_labels, cmap='tab20',                     alpha=0.6, s=30, edgecolors='k', linewidth=0.3)ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} var)', fontsize=11)ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} var)', fontsize=11)ax.set_zlabel(f'PC3 ({pca.explained_variance_ratio_[2]:.1%} var)', fontsize=11)ax.set_title(f'3D PCA Projection of {algorithm} Clusters\n{best_model_info["n_clusters"]} clusters',              fontsize=14, fontweight='bold')plt.colorbar(scatter, ax=ax, label='Cluster', pad=0.1, shrink=0.8)plt.tight_layout()plt.savefig('/home/claude/cluster_3d_visualization.png', dpi=300, bbox_inches='tight')plt.show()print("✓ 3D visualization saved")

## 11. 📊 CLUSTER PROFILING### Detailed Analysis of Each Cluster

In [ ]:
# Add cluster labels to original datadf_clean_with_clusters = df_clean.copy()df_clean_with_clusters['cluster'] = final_labels# Filter out noise points if DBSCANif algorithm == 'DBSCAN':    df_profiling = df_clean_with_clusters[df_clean_with_clusters['cluster'] != -1].copy()else:    df_profiling = df_clean_with_clusters.copy()print("="*80)print("CLUSTER PROFILING")print("="*80)print(f"Total policies: {len(df_profiling)}")print(f"Number of clusters: {df_profiling['cluster'].nunique()}")# Cluster size distributioncluster_sizes = df_profiling['cluster'].value_counts().sort_index()print("\nCluster Size Distribution:")print(cluster_sizes)

In [ ]:
# Profile numerical features by clusternumerical_features_orig = [c for c in pure_numerical if c in df_profiling.columns]print("\n" + "="*80)print("NUMERICAL FEATURE PROFILES BY CLUSTER")print("="*80)for feature in numerical_features_orig[:10]:  # Top 10 numerical features    print(f"\n{feature}:")    profile = df_profiling.groupby('cluster')[feature].agg(['mean', 'median', 'std'])    print(profile)

In [ ]:
# Profile categorical features by clustercategorical_features_orig = [c for c in df_clean.columns                              if df_clean[c].dtype == 'object' and c in df_profiling.columns]print("\n" + "="*80)print("CATEGORICAL FEATURE PROFILES BY CLUSTER")print("="*80)for feature in categorical_features_orig[:5]:  # Top 5 categorical features    print(f"\n{feature}:")    profile = pd.crosstab(df_profiling['cluster'], df_profiling[feature], normalize='index')    print(profile)

In [ ]:
# Visual cluster profilingfig, axes = plt.subplots(2, 2, figsize=(16, 12))# Plot 1: Cluster sizescluster_sizes.plot(kind='bar', ax=axes[0, 0], color='steelblue', edgecolor='black')axes[0, 0].set_xlabel('Cluster', fontsize=12)axes[0, 0].set_ylabel('Number of Policies', fontsize=12)axes[0, 0].set_title('Cluster Size Distribution', fontsize=14, fontweight='bold')axes[0, 0].grid(True, alpha=0.3, axis='y')# Plot 2: Average TIV by clusterif 'policy_tiv' in df_profiling.columns:    tiv_by_cluster = df_profiling.groupby('cluster')['policy_tiv'].mean()    tiv_by_cluster.plot(kind='bar', ax=axes[0, 1], color='coral', edgecolor='black')    axes[0, 1].set_xlabel('Cluster', fontsize=12)    axes[0, 1].set_ylabel('Average Policy TIV', fontsize=12)    axes[0, 1].set_title('Average TIV by Cluster', fontsize=14, fontweight='bold')    axes[0, 1].grid(True, alpha=0.3, axis='y')# Plot 3: Revenue distribution by clusterif 'Revenue' in df_profiling.columns:    df_profiling.boxplot(column='Revenue', by='cluster', ax=axes[1, 0])    axes[1, 0].set_xlabel('Cluster', fontsize=12)    axes[1, 0].set_ylabel('Revenue', fontsize=12)    axes[1, 0].set_title('Revenue Distribution by Cluster', fontsize=14, fontweight='bold')    axes[1, 0].get_figure().suptitle('')# Plot 4: Geographic distributionif 'geo_region' in df_profiling.columns:    region_cluster = pd.crosstab(df_profiling['cluster'], df_profiling['geo_region'], normalize='index')    region_cluster.plot(kind='bar', stacked=True, ax=axes[1, 1], colormap='Set3')    axes[1, 1].set_xlabel('Cluster', fontsize=12)    axes[1, 1].set_ylabel('Proportion', fontsize=12)    axes[1, 1].set_title('Geographic Distribution by Cluster', fontsize=14, fontweight='bold')    axes[1, 1].legend(title='Region', bbox_to_anchor=(1.05, 1), loc='upper left')plt.tight_layout()plt.savefig('/home/claude/cluster_profiling.png', dpi=300, bbox_inches='tight')plt.show()print("\n✓ Cluster profiling visualization saved")

In [ ]:
# Generate comprehensive cluster reportprint("\n" + "="*80)print("COMPREHENSIVE CLUSTER REPORT")print("="*80)for cluster_id in sorted(df_profiling['cluster'].unique()):    cluster_data = df_profiling[df_profiling['cluster'] == cluster_id]        print(f"\n{'='*80}")    print(f"CLUSTER {cluster_id}")    print(f"{'='*80}")    print(f"Size: {len(cluster_data)} policies ({len(cluster_data)/len(df_profiling)*100:.1f}%)")        # Key numerical characteristics    if 'policy_tiv' in cluster_data.columns:        print(f"\nAverage TIV: ${cluster_data['policy_tiv'].mean():,.0f}")    if 'Revenue' in cluster_data.columns:        print(f"Average Revenue: ${cluster_data['Revenue'].mean():,.0f}")    if 'EMP_TOT' in cluster_data.columns:        print(f"Average Employees: {cluster_data['EMP_TOT'].mean():.0f}")        # Dominant categories    if 'Product' in cluster_data.columns:        top_product = cluster_data['Product'].mode()[0] if len(cluster_data['Product'].mode()) > 0 else 'N/A'        print(f"\nMost Common Product: {top_product}")        if 'Portfolio Segmentation' in cluster_data.columns:        top_segment = cluster_data['Portfolio Segmentation'].mode()[0] if len(cluster_data['Portfolio Segmentation'].mode()) > 0 else 'N/A'        print(f"Most Common Segment: {top_segment}")        if 'geo_region' in cluster_data.columns:        top_region = cluster_data['geo_region'].mode()[0] if len(cluster_data['geo_region'].mode()) > 0 else 'N/A'        print(f"Most Common Region: {top_region}")print("\n✓ Cluster profiling complete")

## 12. Build Similarity Engine Based on Best Clustering Model

In [ ]:
# Build hybrid similarity engine using insights from clusteringclass HybridSimilarityEngine:    '''Production similarity engine informed by clustering analysis'''        def __init__(self, n_neighbors=3, use_faiss=False):        self.n_neighbors = n_neighbors        self.use_faiss = use_faiss and FAISS_AVAILABLE        self.model = None        self.X_train = None        self.cluster_labels = None            def fit(self, X, cluster_labels=None):        '''Fit similarity engine'''        self.X_train = X        self.cluster_labels = cluster_labels                if self.use_faiss:            self.model = faiss.IndexFlatL2(X.shape[1])            self.model.add(X.astype('float32'))            print("  Using FAISS for approximate search")        else:            self.model = NearestNeighbors(                n_neighbors=self.n_neighbors,                metric='euclidean',                n_jobs=-1            )            self.model.fit(X)            print("  Using sklearn for exact search")                return self        def find_similar(self, query_vector, same_cluster_only=False, query_cluster=None):        '''Find similar policies'''                if self.use_faiss:            distances, indices = self.model.search(                query_vector.reshape(1, -1).astype('float32'),                self.n_neighbors * 3 if same_cluster_only else self.n_neighbors            )            indices, distances = indices[0], distances[0]        else:            distances, indices = self.model.kneighbors(                query_vector.reshape(1, -1),                n_neighbors=self.n_neighbors * 3 if same_cluster_only else self.n_neighbors            )            indices, distances = indices[0], distances[0]                # Filter by cluster if requested        if same_cluster_only and self.cluster_labels is not None and query_cluster is not None:            mask = self.cluster_labels[indices] == query_cluster            indices = indices[mask][:self.n_neighbors]            distances = distances[mask][:self.n_neighbors]                return indices, distances# Initialize final similarity enginesimilarity_engine = HybridSimilarityEngine(    n_neighbors=3,    use_faiss=FAISS_AVAILABLE)print("Fitting similarity engine...")similarity_engine.fit(X_pca_95, final_labels)print("✓ Similarity engine fitted")

## 13. Validation & Explainability with SHAP

In [ ]:
# Test similarity retrievaltest_policy_idx = 100similar_indices, distances = similarity_engine.find_similar(X_pca_95[test_policy_idx])print("="*80)print("SIMILARITY RETRIEVAL TEST")print("="*80)print(f"Query Policy Index: {test_policy_idx}")print(f"Query Policy Cluster: {final_labels[test_policy_idx]}")print(f"\nTop 3 Similar Policies:")for i, (idx, dist) in enumerate(zip(similar_indices, distances)):    policy_id = identifiers.iloc[idx]['System Reference Number']    cluster_id = final_labels[idx]    score = 1 / (1 + dist)    print(f"  {i+1}. Policy {policy_id}")    print(f"     Cluster: {cluster_id} | Distance: {dist:.4f} | Score: {score:.4f}")

In [ ]:
# SHAP Explainability (if available)if SHAP_AVAILABLE:    print("\n" + "="*80)    print("SHAP EXPLAINABILITY ANALYSIS")    print("="*80)        # Train a simple model as surrogate for SHAP    from sklearn.ensemble import RandomForestClassifier        print("Training surrogate model for SHAP...")    surrogate_model = RandomForestClassifier(        n_estimators=100,        max_depth=10,        random_state=RANDOM_SEED,        n_jobs=-1    )        # Sample for faster computation    sample_size = min(1000, len(X_pca_95))    sample_idx = np.random.choice(len(X_pca_95), sample_size, replace=False)    X_sample = X_pca_95[sample_idx]    y_sample = final_labels[sample_idx]        surrogate_model.fit(X_sample, y_sample)    print("✓ Surrogate model trained")        # SHAP TreeExplainer    print("Computing SHAP values...")    explainer = shap.TreeExplainer(surrogate_model)    shap_values = explainer.shap_values(X_sample[:100])  # First 100 samples        # Plot summary    print("Generating SHAP summary plot...")    plt.figure(figsize=(12, 8))    shap.summary_plot(        shap_values,        X_sample[:100],        feature_names=[f'PC{i+1}' for i in range(X_pca_95.shape[1])],        show=False    )    plt.title('SHAP Feature Importance for Clustering', fontsize=14, fontweight='bold')    plt.tight_layout()    plt.savefig('/home/claude/shap_summary.png', dpi=300, bbox_inches='tight')    plt.show()        print("✓ SHAP analysis complete")else:    print("\n⚠️ SHAP not available - skipping explainability analysis")    print("   Install with: pip install shap")

In [ ]:
# Feature importance from PCAprint("\n" + "="*80)print("PCA COMPONENT IMPORTANCE")print("="*80)# Get feature contributions to top componentsn_top_components = 5feature_names_orig = df_scaled.columns.tolist()for i in range(n_top_components):    print(f"\nPrincipal Component {i+1} (Explains {pca.explained_variance_ratio_[i]:.2%} variance):")        # Get loadings    loadings = pca.components_[i]        # Top positive contributors    top_positive_idx = np.argsort(loadings)[-5:][::-1]    print("  Top Positive Contributors:")    for idx in top_positive_idx:        print(f"    {feature_names_orig[idx]}: {loadings[idx]:.4f}")        # Top negative contributors    top_negative_idx = np.argsort(loadings)[:5]    print("  Top Negative Contributors:")    for idx in top_negative_idx:        print(f"    {feature_names_orig[idx]}: {loadings[idx]:.4f}")

## 14. Model Serialization

In [ ]:
# Create models directoryos.makedirs('/home/claude/models', exist_ok=True)print("="*80)print("SAVING ALL MODELS AND ARTIFACTS")print("="*80)# 1. Clustering modeljoblib.dump(final_model, '/home/claude/models/clustering_model.pkl')joblib.dump(final_labels, '/home/claude/models/cluster_labels.pkl')print("✓ Clustering model saved")# 2. Similarity enginejoblib.dump(similarity_engine, '/home/claude/models/similarity_engine.pkl')print("✓ Similarity engine saved")# 3. Preprocessing componentsjoblib.dump(scaler, '/home/claude/models/scaler.pkl')joblib.dump(pca, '/home/claude/models/pca_model.pkl')joblib.dump(frequency_encodings, '/home/claude/models/frequency_encodings.pkl')print("✓ Preprocessing components saved")# 4. Text embedding model reference (if used)if text_embeddings:    joblib.dump(text_embeddings, '/home/claude/models/text_embeddings.pkl')    print("✓ Text embeddings saved")# 5. Feature metadatametadata = {    'feature_names': df_scaled.columns.tolist(),    'pure_numerical': pure_numerical,    'low_cardinality': low_cardinality,    'high_cardinality': high_cardinality,    'text_fields': text_fields,    'n_pca_components': n_components_95,    'random_seed': RANDOM_SEED,    'model_version': '2.0.0',    'training_date': datetime.now().isoformat(),    'best_clustering_algorithm': algorithm,    'best_clustering_params': params,    'n_clusters': int(best_model_info['n_clusters']),    'best_silhouette_score': float(best_model_info['silhouette']),    'best_davies_bouldin': float(best_model_info['davies_bouldin']),    'best_calinski_harabasz': float(best_model_info['calinski_harabasz'])}joblib.dump(metadata, '/home/claude/models/metadata.pkl')with open('/home/claude/models/metadata.json', 'w') as f:    json.dump(metadata, f, indent=2)print("✓ Metadata saved")# 6. Identifiersidentifiers.to_csv('/home/claude/models/policy_identifiers.csv', index=False)print("✓ Identifiers saved")# 7. Cluster assignmentsdf_with_clusters = identifiers.copy()df_with_clusters['cluster'] = final_labelsdf_with_clusters.to_csv('/home/claude/models/policy_clusters.csv', index=False)print("✓ Cluster assignments saved")# 8. All clustering resultsall_results.to_csv('/home/claude/models/clustering_evaluation_results.csv', index=False)print("✓ Evaluation results saved")print("\n" + "="*80)print("✅ ALL MODELS SAVED SUCCESSFULLY")print("="*80)print(f"Location: /home/claude/models/")print(f"Files saved: {len(os.listdir('/home/claude/models/'))} artifacts")

## 15. Summary & Deployment Readiness### 🎉 Training Pipeline Complete!#### Models Trained:✅ **Clustering Analysis** - Evaluated {len(all_results)} algorithm configurations  ✅ **Best Model Selected** - {algorithm} with optimal hyperparameters  ✅ **Similarity Engine** - Production-ready retrieval system  ✅ **Full Validation** - Feature matrix validated, clusters profiled  ✅ **Explainability** - SHAP analysis and PCA interpretation  #### Key Metrics:- **Silhouette Score**: {best_model_info['silhouette']:.4f}- **Davies-Bouldin Index**: {best_model_info['davies_bouldin']:.4f}- **Number of Clusters**: {best_model_info['n_clusters']}- **Combined Quality Score**: {best_model_info['combined_score']:.4f}#### Deliverables:✓ Trained clustering model  ✓ Similarity retrieval engine  ✓ PCA dimensionality reduction  ✓ Feature encoders and scalers  ✓ Comprehensive cluster profiles  ✓ Visualization artifacts  ✓ Evaluation metrics  ### Next Steps:1. **Load Models** - Use prediction notebook2. **Deploy API** - Create REST endpoint3. **Monitor Performance** - Track retrieval quality4. **Retrain** - Quarterly or when PSI > 0.25---**Model Version**: 2.0.0  **Status**: ✅ Production Ready  **Training Date**: {datetime.now().strftime('%Y-%m-%d')}---